<a href="https://colab.research.google.com/github/emni8523/neural_networks_project/blob/main/backend/speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/data'
!ls "$data_path"

import pandas as pd

df = pd.read_parquet(data_path + '/test-00000-of-00001.parquet')
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
output			      train-00001-of-00003.parquet
test-00000-of-00001.parquet   train-00002-of-00003.parquet
train-00000-of-00003.parquet  validation-00000-of-00001.parquet


,audio,transcript_annotated,transcript_a,transcript_b,transcript_c
0,{'bytes': b'RIFF<G\x05\x00WAVEfmt \x10\x00\x00...,"Yeah. <laughter> {D Well, } rabbits are darlin...","Yeah. Well, rabbits are darling. That would be...",Yeah. rabbits are darling. That would be fun i...,Yeah. rabbits are darling. That would be fun i...
1,{'bytes': b'RIFF\x04r\x03\x00WAVEfmt \x10\x00\...,"{D Well, } McNeil , [ Neil-, + ] and Lehrer, ...","Well, McNeil. Neil-, and Lehrer, I guess he's ...","McNeil, Neil-, and Lehrer, I guess he's leavin...","McNeil, and Lehrer, I guess he's leaving, one ..."
2,{'bytes': b'RIFF\xe4\x9f\x04\x00WAVEfmt \x10\x...,"{D Well, } [ I don't want to, + I don't want t...","Well, I don't want to, I don't want this to be...","I don't want to, I don't want this to be a uni...",I don't want this to be a unilateral conversat...
3,{'bytes': b'RIFF\xd4\xcd\x04\x00WAVEfmt \x10\x...,"{D Well, } what about, {F uh, } papyrus, {D yo...","Well, what about, uh, papyrus, you know, made ...","what about, papyrus, made out of bam-, bamboo ...","what about, papyrus, made out of bamboo stuff,..."
4,{'bytes': b'RIFF(\xe4\x03\x00WAVEfmt \x10\x00\...,"{C Because } women are in every field now, {E ...","Because women are in every field now, I mean I...","Because women are in every field now, I can't ...","Because women are in every field now, I can't ..."


In [ ]:
import numpy as np
import librosa
%pip install transformers accelerate datasets

In [ ]:
from IPython.core.interactiveshell import dis
import pandas as pd
from pathlib import Path
import os
import random


def load_disfluency_dataset(filePath, output_path, split_ratio=(0.7, 0.15, 0.15),
                            seed=42):

  random.seed(seed)
  data_list = []
  disfluency_path = Path(filePath)
  output_base_path = Path(output_path)


  for root, dirs, files in os.walk(disfluency_path):

    root_path = Path(root)

    # Create output directories for audio files
    audio_dir = output_base_path / "audio"
    audio_dir.mkdir(parents=True, exist_ok=True)

    index = 0
    for f in files:
      file_path = root_path / f
      print(f'Reading from file: {file_path}')

      data = pd.read_parquet(file_path)
      data = data[['audio', 'transcript_a']]
      data = data.rename(columns={'transcript_a': 'transcript'})

      for idx, row in data.iterrows():
        audio_data = row['audio']
        transcript = row['transcript']

        if isinstance(audio_data, dict) and 'bytes' in audio_data:
            audio_bytes = audio_data['bytes']
        else:
            print(f"Skipping row {idx}: audio not in expected format")
            continue

        # Saving the bytes
        audio_filename = f"audio_{index:06d}.wav"
        audio_path = audio_dir / audio_filename

        with open(audio_path, 'wb') as audio_file:
            audio_file.write(audio_bytes)

        data_list.append({
            'id': index,
            'audio_path': str(audio_path),
            'transcript': transcript
        })

        index = index + 1

    random.shuffle(data_list)
    n = len(data_list)

    if n == 0:
        return {'train': pd.DataFrame(), 'val': pd.DataFrame(), 'test': pd.DataFrame()}

    n_train = int(split_ratio[0] * n)
    n_val = int(split_ratio[1] * n)

    train_df = pd.DataFrame(data_list[:n_train])
    val_df = pd.DataFrame(data_list[n_train:n_train + n_val])
    test_df = pd.DataFrame(data_list[n_train + n_val:])

    train_df.to_csv(output_base_path / "train.csv", index=False)
    val_df.to_csv(output_base_path / "val.csv", index=False)
    test_df.to_csv(output_base_path / "test.csv", index=False)

    print(f"Split into: {len(train_df)} train, {len(val_df)} val, {len(test_df)} test")

    return {'train': train_df, 'val': val_df, 'test': test_df}


output_path = '/content/drive/MyDrive/data/output'

disfleuncy_dataset = load_disfluency_dataset(data_path, output_path)


Streaming output truncated to the last 5000 lines.
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Worki

In [ ]:


from pathlib import Path
output_path = '/content/drive/MyDrive/data/output'
output_path = Path(output_path)
already_runned = True


#Saving the info to csv files
if already_runned:
  disfleuncy_dataset = {
    'train': pd.read_csv(output_path / "train.csv"),
    'val': pd.read_csv(output_path / "val.csv"),
    'test': pd.read_csv(output_path / "test.csv")}
else:
  print("Have to load in the dataset first")
print(disfleuncy_dataset['train'].head())




     id                                         audio_path  \
0  1383  /content/drive/MyDrive/data/output/audio/audio...   
1  1881  /content/drive/MyDrive/data/output/audio/audio...   
2  4901  /content/drive/MyDrive/data/output/audio/audio...   
3  1524  /content/drive/MyDrive/data/output/audio/audio...   
4  4654  /content/drive/MyDrive/data/output/audio/audio...   

                                          transcript  
0  Because they found out later on that, jeez, yo...  
1  I mean, I, uh, you know, I, I talked to a few ...  
2  Yeah, it's still, I mean, you know, two thousa...  
3  And then you put all your lar-, yard clippings...  
4  Oh, you think, children, you can, um, use craf...  


In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [ ]:
from datasets import Dataset

def prepare_dataset(batch):

  # Load audio
  audio, sr = librosa.load(batch['audio_path'], sr=16000)

  # The log mel spectrum
  batch["input_features"] = processor.feature_extractor(audio, sr,
                            return_tensors="pt").input_features[0]

  # Tokenize
  batch["labels"] = tokenizer(batch["transcript"]).input_ids

  return batch

train_df = disfleuncy_dataset['train']
test_df = disfleuncy_dataset['test']

# Convert to Dataset objects (For huggingface)
disfluency_train = Dataset.from_pandas(train_df)
disfluency_test = Dataset.from_pandas(test_df)

print("Starting the formating")
disfluency_train = disfluency_train.map(
    prepare_dataset,
    remove_columns=disfluency_train.column_names,
    num_proc=1
)

disfluency_test = disfluency_test.map(
    prepare_dataset,
    remove_columns=disfluency_test.column_names,
    num_proc=1
)


print(disfluency_train[0].keys())

{'id': 1383, 'audio_path': '/content/drive/MyDrive/data/output/audio/audio_001383.wav', 'transcript': 'Because they found out later on that, jeez, you know, a guy will lie rather than get killed.'}
Starting the formating


Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

dict_keys(['input_features', 'labels'])


In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small",
                                             language="English",
                                             task="transcribe")


model.generation_config.language = "english"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=processor.tokenizer,
    model=model,
    padding=True,
)


In [ ]:
%pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 46.9 MB/s eta 0:00:00


In [ ]:
from torchmetrics.text import WordErrorRate


wer = WordErrorRate()

def compute_metrics(pred):
  pred_ids = pred.predictions
  label_ids = pred.label_ids

  # Replace ignore index used by HF with pad token
  label_ids[label_ids == -100] = tokenizer.pad_token_id

  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

  wer_value = wer(pred_str, label_str).item() * 100

  return {"wer": wer_value}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/whisper-small-fineTuned-disfluency",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=0.01,
    fp16=True,
    report_to="none",
    save_strategy="epoch",
    save_total_limit=1,
    logging_strategy="epoch",
)

In [ ]:
import transformers
import accelerate
import datasets
import torch

print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("datasets:", datasets.__version__)
print("torch:", torch.__version__)

transformers: 4.57.1
accelerate: 1.11.0
datasets: 4.0.0
torch: 2.8.0+cu126


In [ ]:
from transformers import Trainer

trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=disfluency_train,
    eval_dataset=disfluency_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer
)

NameError: name 'disfluency_train' is not defined

In [ ]:
trainer.train()
trainer.save_model()

Step,Training Loss
875,0.016800
1750,0.007800
2625,0.002900
3500,0.000800
4375,0.000400


TrainOutput(global_step=4375, training_loss=0.005739203446252006, metrics={'train_runtime': 3188.8536, 'train_samples_per_second': 5.488, 'train_steps_per_second': 1.372, 'total_flos': 5.0502445056e+18, 'train_loss': 0.005739203446252006, 'epoch': 5.0})

In [ ]:
%pip install evaluate jiwer torchinfo

In [ ]:
from transformers import pipeline, WhisperFeatureExtractor
from evaluate import load
import soundfile as sf
from torchinfo import summary
import random

# Load WER metric
wer_metric = load("wer")


feature_extractor_pipeline = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


# My model
asr = pipeline(task="automatic-speech-recognition",
    model="/content/drive/MyDrive/whisper-small-fineTuned-disfluency/checkpoint-4375",
    feature_extractor=feature_extractor_pipeline
    )


# Open whispers model
real_model = pipeline(task="automatic-speech-recognition",
   model="openai/whisper-small",
    feature_extractor=feature_extractor_pipeline)


def transcribe(audio):
    text_asr = asr(audio)["text"]

    text_real_model = real_model(audio)["text"]

    return text_asr, text_real_model


prediction_asr = []
prediction_whisper = []
reference = []



def get_duration(path):
    try:
        with sf.SoundFile(path) as f:
            return len(f) / f.samplerate
    except:
        return None

disfluency_val = disfleuncy_dataset['val']

for i in range(len(disfluency_val)):

  random_val = random.randint(0, len(disfluency_val) - 1)
  sample = disfluency_val.iloc[random_val]

  audio_path = sample['audio_path']
  duration = get_duration(audio_path)
  if duration >= 30:
    print("-" * 50)
    print("WARNING")
    print(f'sample: {sample['transcript']}')
    print(f'Had a Duration of {duration}')
    print("-" * 50)
    continue

  print(f"Sample {i + 1}:")
  print(f"Transcript: {sample['transcript']}")
  reference.append(sample['transcript'])


  asr_output, whisper_output = transcribe(audio_path)


  print(f"ASR output: {asr_output}")
  prediction_asr.append(asr_output)
  print()
  print(f"Whisper output: {whisper_output}")
  print()
  prediction_whisper.append(whisper_output)

  asr_wer = wer_metric.compute(predictions=[asr_output],
                                      references=[sample['transcript']])
  print(f"WER ASR: {asr_wer * 100:.2f}%")
  print("-" * 50)

  whisper_wer = wer_metric.compute(predictions=[whisper_output],
                                      references=[sample['transcript']])
  print(f"WER Whisper: {whisper_wer * 100:.2f}%")
  print("-" * 50)

average_wer_asr = wer_metric.compute(predictions=prediction_asr, references=reference)
average_wer_whisper = wer_metric.compute(predictions=prediction_whisper, references=reference)
print(f"\n{'='*50}")
print(f"Average WER for ASR: {average_wer_asr * 100:.2f}%")
print(f"Average WER for Whisper: {average_wer_whisper * 100:.2f}%")
print(f"{'='*50}")








Device set to use cuda:0
Device set to use cuda:0


Sample 1:
Transcript: But, uh, I try, a-, as with the American Express, I try to pay it, you know, as it comes in.
ASR output: But, uh, I try, uh, as with the American Express, I try to pay it, you know, as it comes in.

Whisper output:  But I try, as with the American Express, I try to pay it, you know, as it comes in.

WER ASR: 4.76%
--------------------------------------------------
WER Whisper: 14.29%
--------------------------------------------------
Sample 2:
Transcript: I had water coming in everything that was in the ceiling, everywhere. And I mean it filled up my light fixtures, because they're the old kind.
ASR output: I had water coming in everything that was in the ceiling, everywhere, and, I mean, it filled up my light fixtures, because they're the old kind.

Whisper output:  I had water coming in everything that was in the ceiling, everywhere. And I mean, it filled up my light fixtures because they are the old kind.

WER ASR: 11.54%
---------------------------------------

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.

WER ASR: 12.12%
--------------------------------------------------
WER Whisper: 18.18%
--------------------------------------------------
Sample 252:
Transcript: And I, the, the job that I do, I do a lot of training of our customers for new products.
ASR output: And I, the, the job that I do, I do a lot of training of our customers for new products.

Whisper output:  And the job that I do, I do a lot of training of our customers for new products.

WER ASR: 0.00%
--------------------------------------------------
WER Whisper: 10.00%
--------------------------------------------------
Sample 253:
Transcript: Right and that, and, you know, also by keeping them in prison, you do have the possibility, though, we don't currently do this, of making restitution.
ASR output: Right, and that, and, you know, also by keeping them in prison you do have the possibility, though we don't currently do this, of making restitution.

Whisper output:  Righ

In [ ]:

print(f"\nArchitecture:")
print(f"Hidden size: {asr.model.config.d_model}")
print(f"Encoder layers: {asr.model.config.encoder_layers}")
print(f"Decoder layers: {asr.model.config.decoder_layers}")
print(f"Attention heads: {asr.model.config.encoder_attention_heads}")
print(f"Vocab size: {asr.model.config.vocab_size}")



Architecture:
Hidden size: 768
Encoder layers: 12
Decoder layers: 12
Attention heads: 12
Vocab size: 51865
